In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
import os
import errno
import utils

import CGDs
import importlib
importlib.reload(CGDs)


from pyDOE import lhs
from torch import from_numpy

import torch
import torch.cuda
import torch.nn as nn
import torch.optim as optim
import torch.autograd as tgrad

# Manage device

In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())
# torch.set_default_tensor_type(torch.DoubleTensor)
print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

True
cuda


In [3]:
utils.printMemory()

total: 25756696576, reserved: 0, free: 0


# Data Sampling

In [4]:
samples = {"pde": 50000, "bc":5000, "fc":5000}

K = 40.0
r = 0.05
sigma = 0.25
T = 1.0
S_range = [0.0, 130.0]
t_range = [0.0, T]
gs = lambda x: np.fmax(x-K, 0.0)

# Build Neural Network

In [5]:
import networks
layers = np.array([2, 50, 50, 50, 1])

# Create the model
PINNBCGD = networks.FeedforwardNeuralNetwork(2, 50, 1, 3)
PINNBCGD.to(device)
print(PINNBCGD)

FeedforwardNeuralNetwork(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1-2): 2 x Linear(in_features=50, out_features=50, bias=True)
  )
  (output): Linear(in_features=50, out_features=1, bias=True)
  (relu): ReLU()
)


In [6]:
D_BCGD = networks.Discriminator(2, 25, 1)
D_BCGD.to(device)
D_BCGD.load_state_dict(D_BCGD.state_dict()) # copy weights and stuff
print(D_BCGD)

Discriminator(
  (map): Sequential(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): ReLU()
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): ReLU()
    (8): Linear(in_features=50, out_features=1, bias=True)
  )
)


# Network Trainig

### Hyperparameters

In [7]:
max_iter = 60000
graphPer = 0
iter_recordBCGD = 200

savePer = 60000

# Define loss function and optimizer
optimizer = CGDs.BCGD(max_params=D_BCGD.parameters(), min_params= PINNBCGD.parameters(), device = device,
                 lr_max=3e-5, lr_min=3e-5, tol=1e-10, collect_info=True)
lossFunction = torch.nn.MSELoss()

### Training

In [8]:
start_time=time.time()
loss_hist = []

for epoch in range(max_iter):
    
    optimizer.zero_grad() # zeroes the gradient buffers of all parameters
    
    # sampling
    bc_st_train, bc_v_train, n_st_train, n_v_train = \
    utils.trainingData(K, 
                       r, 
                       sigma, 
                       T, 
                       S_range[-1], 
                       S_range, 
                       t_range, 
                       gs, 
                       samples['bc'], 
                       samples['fc'], 
                       samples['pde'], 
                       RNG_key=123)
    
    # save training data points to tensor and send to device
    n_st_train = torch.from_numpy(n_st_train).float().requires_grad_().to(device)
    n_v_train = torch.from_numpy(n_v_train).float().to(device)
    
    bc_st_train = torch.from_numpy(bc_st_train).float().to(device)
    bc_v_train = torch.from_numpy(bc_v_train).float().to(device)
    
    
    # normal loss
    # print(n_st_train)
    # print(PINNBCGD.output.weight.dtype)
    v1_hat = PINNBCGD(n_st_train)
    
    grads = tgrad.grad(v1_hat, n_st_train, grad_outputs=torch.ones(v1_hat.shape).cuda(), 
                       retain_graph=True, create_graph=True, only_inputs=True)[0]
    dVdt, dVdS = grads[:, 0].view(-1, 1), grads[:, 1].view(-1, 1)
    grads2nd = tgrad.grad(dVdS, n_st_train, grad_outputs=torch.ones(dVdS.shape).cuda(), create_graph=True, only_inputs=True)[0]
    d2VdS2 = grads2nd[:, 1].view(-1, 1)
    S1 = n_st_train[:, 1].view(-1, 1)
    pde_loss = lossFunction(-dVdt, 0.5*((sigma*S1)**2)*d2VdS2 + r*S1*dVdS - r*v1_hat)
    
    D_output1 = D_BCGD(n_st_train)
    loss1 = D_output1 * pde_loss
    
    
    # boundary condition loss
    bc_hat = PINNBCGD(bc_st_train)
    bc_loss = lossFunction(bc_v_train, bc_hat)
    
    loss2 = D_BCGD(bc_st_train) * bc_loss
    
    
    # Backpropagation and Update
    combined_loss = loss1.mean() + loss2.mean()
    pinn_loss = pde_loss.mean() + bc_loss.mean()
    # combined_loss.backward()
    optimizer.step(combined_loss)
    
    loss_hist.append(combined_loss.item())
    if epoch % 500 == 0:
        print(f'{epoch}/{max_iter} PDE Loss: {pde_loss.item():.5f}, BC Loss: {bc_loss.item():.5f}, total loss: {pinn_loss.item():5f}, neural network loss: {combined_loss.item():5f}')
        pass
        
end_time = time.time()
print('run time:', end_time - start_time)
print('finish')

0/60000 PDE Loss: 0.00067, BC Loss: 3559.49292, total loss: 3559.493652, neural network loss: 1333.793213
500/60000 PDE Loss: 0.13118, BC Loss: 101.86180, total loss: 101.992981, neural network loss: 145.076080
1000/60000 PDE Loss: 0.21456, BC Loss: 98.79087, total loss: 99.005432, neural network loss: 146.534714


KeyboardInterrupt: 